In [53]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [2]:
train_data = pd.read_excel("Train.xlsx")
test_data = pd.read_excel("Test.xlsx")


In [40]:
data = pd.concat([train_data, test_data], axis=0)

In [42]:
print(data.columns)


Index(['timestamp', 'Temperature', 'Precipitation', 'Snowfall',
       'Totalcloudiness', 'Highcloudcover', 'Mediumcloudcover',
       'Lowcloudcover', 'Pressure', 'Windspeedm/s', 'WindDirection',
       'WindGusts'],
      dtype='object')


In [43]:
input_parameters = ["timestamp"]
output_parameters = ["Temperature", "Precipitation", "Snowfall", "Totalcloudiness","Lowcloudcover", "Mediumcloudcover", "Highcloudcover","Pressure", "Windspeedm/s", "WindDirection", "WindGusts"]


In [44]:
data["timestamp"] = pd.to_datetime(data["timestamp"], format="%Y-%m-%dT%H:%M:%S")
data.set_index("timestamp", inplace=True)
data.sort_index(inplace=True)


In [45]:
data.fillna(method="ffill", inplace=True)


In [46]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)


In [47]:
# Number of previous timestamps to consider as input for predicting the next timestamp
window_size = 24
X = []
y = []
for i in range(len(data) - window_size):
    X.append(scaled_data[i:i + window_size])
    y.append(scaled_data[i + window_size])

X = np.array(X)
y = np.array(y)


In [48]:
train_size = len(train_data)
X_train = X[:train_size]
y_train = y[:train_size]
X_test = X[train_size:]
y_test = y[train_size:]


In [49]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True,input_shape=(window_size, len(data.columns))))
model.add(LSTM(units=50))
model.add(Dense(len(output_parameters)))


In [50]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [51]:
model.fit(X_train, y_train, epochs=100, batch_size=32,validation_data=(X_test, y_test))


Epoch 1/100
1209/1209 [==============================] - 21s 16ms/step - loss: 0.0307 - val_loss: 0.0221
Epoch 2/100
1209/1209 [==============================] - 19s 16ms/step - loss: 0.0245 - val_loss: 0.0210
Epoch 3/100
1209/1209 [==============================] - 18s 15ms/step - loss: 0.0241 - val_loss: 0.0211
Epoch 4/100
1209/1209 [==============================] - 22s 18ms/step - loss: 0.0239 - val_loss: 0.0209
Epoch 5/100
1209/1209 [==============================] - 22s 18ms/step - loss: 0.0238 - val_loss: 0.0210
Epoch 6/100
1209/1209 [==============================] - 16s 13ms/step - loss: 0.0237 - val_loss: 0.0207
Epoch 7/100
1209/1209 [==============================] - 17s 14ms/step - loss: 0.0235 - val_loss: 0.0212
Epoch 8/100
1209/1209 [==============================] - 22s 18ms/step - loss: 0.0235 - val_loss: 0.0207
Epoch 9/100
1209/1209 [==============================] - 22s 18ms/step - loss: 0.0234 - val_loss: 0.0204
Epoch 10/100
1209/1209 [==============================]

In [52]:
predictions = model.predict(X_test)

predictions = scaler.inverse_transform(predictions)
output_dates = data.index[train_size + window_size:].to_list()
output_df = pd.DataFrame(predictions, columns=output_parameters)
output_df["timestamp"] = output_dates
output_df.set_index("timestamp", inplace=True)
print(output_df)


22/22 [==============================] - 0s 4ms/step
                     Temperature  Precipitation  Snowfall  Totalcloudiness  \
timestamp                                                                    
2023-06-02 00:00:00    23.629255       0.001321 -0.032493        41.234798   
2023-06-02 01:00:00    21.037395       0.018785 -0.036330        60.129238   
2023-06-02 02:00:00    18.373285       0.018493  0.000372        55.344486   
2023-06-02 03:00:00    17.234438       0.015557 -0.010401        78.325211   
2023-06-02 04:00:00    14.992988      -0.033911 -0.015247        28.106016   
...                          ...            ...       ...              ...   
2023-06-30 19:00:00    21.916382       0.002984 -0.011210        11.322035   
2023-06-30 20:00:00    23.256220       0.030522  0.015275        82.952942   
2023-06-30 21:00:00    21.796694      -0.001396  0.002007        18.627501   
2023-06-30 22:00:00    20.428364       0.007380  0.014906        14.852391   
2023-06-30 

In [56]:
output_df.index = test_data.index[:len(output_df)]

# Calculate errors
mse = mean_squared_error(
    test_data[output_parameters][:len(output_df)], output_df)
mae = mean_absolute_error(
    test_data[output_parameters][:len(output_df)], output_df)
rmse = np.sqrt(mse)

# Print the errors
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)


Mean Squared Error (MSE): 1843.190833030358
Mean Absolute Error (MAE): 17.713046051816544
Root Mean Squared Error (RMSE): 42.932398407617036
